## Documentation:


###  Objective:
Find variables that helps to explain product Salesrank in Amazon

### What things this notebook has done?
1. Change tabular data into item-wise dictionary

2. Metrics generation:
   1. Calculation: aggregation metrics (Average ratings)
   2. Date: time metrics
   3. Text: 
       average word count
      

3. Regression basedline model


### What's the dataset about?
1. Avery's product reviews data from Amazon
2. Sales Rank of Avery products under different categories


### Run time


### Common bugs:


LOL

### Below are detailed code chunks 

## Part 1: Setup working environment & Dataset
* Load sales rank & review dataset for Avery product

In [233]:
debug=True
option=2

In [234]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

#!pip install quilt
#!quilt install ResidentMario/missingno_data
from quilt.data.ResidentMario import missingno_data

%matplotlib inline

In [235]:
# ar1 = pd.read_csv('Avery_reviews.csv')
# asr1 = pd.read_csv('SalesRank_Avery.csv')
# ap1 = pd.read_csv('Avery_price.csv')
ar1 = pd.read_csv('FOP_reviews.csv')
asr1 = pd.read_csv('FOP_salesrank.csv')
ap1 = pd.read_csv('FOP_price.csv')

/Users/jiaxuewan/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/jiaxuewan/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Options for modeling:
* Group 1: more variability
* Group 2: most reviews records

Must under missing_value check

In [236]:

if(option==1):
    # more reviews and highly variant sales rank
    common = ['B002A9JQSG',
 'B00GMOOB08',
 'B00KP37YW4',
 'B01CZNLK12',
 'B074JK3XHC',
 'B07BKVN8S4']
    
elif(option==2):
    # products with most reviews
    common=['B000WB397I',
 'B000YGO7HW',
 'B0026I2HLO',
 'B00FNK8UQS',
 'B010E9M3KG',
 'B074QHZJ9K']
else:
    pass


In [237]:

if(debug):
    asr=asr1[asr1['id'].isin(common)].set_index('id',drop=True)
    #asr1 = asr1.set_index('id',drop = True)
    ar=ar1[ar1['product'].isin(common)].set_index('product',drop=True)
    ap=ap1[ap1['id'].isin(common)].set_index('id',drop=True)
else:
    asr = asr1.set_index('id',drop = True)
    # Same as above, create another dictionary for review data
    ar = ar1.set_index('product',drop = True)
    ap = ap1.set_index('id',drop=True)

### Transform into key-value pairs by each unique product ID
To make aggregation on review ratings, we need details on each product



In [238]:
# create a dictionary to store the information for each product. Each product represents one element of the dictionary.
asr_dict = {}
for i in asr.index.unique():
    asr_dict[i] = asr.loc[i, ['source','date', 'category_id1',
       'category_name1', 'category_rank1', 'category_id2', 'category_name2',
       'category_rank2', 'category_id3', 'category_name3', 'category_rank3',
       'category_id4', 'category_name4', 'category_rank4']]

In [239]:
item_list=list(asr_dict.keys())
item_list[1]

'B010E9M3KG'


### Features Time based rank Features
* log_rank 
* delta_rank
* date

#### Salesrank dataset

In [240]:
for i in asr_dict:
    if type(asr_dict[i]) != pd.core.series.Series:
        asr_dict[i] = asr_dict[i].sort_values('date').reset_index()
        asr_dict[i].drop_duplicates(inplace = True)
        asr_dict[i]['new_date'] = pd.to_datetime(asr_dict[i]['date'])
        asr_dict[i]['log_rank'] = asr_dict[i]['category_rank1'].map(lambda x: math.log(x))
        asr_dict[i]['delta_rank'] = asr_dict[i]['log_rank'].diff()
        a = asr_dict[i].index[0]
        asr_dict[i].drop(a, inplace = True)
        asr_dict[i]['new_date'] = asr_dict[i]['new_date'].dt.strftime('%Y-%m-%d')
        asr_dict[i]['new_date'] = pd.to_datetime(asr_dict[i]['new_date'])


In [241]:
for i in asr_dict:
    if type(asr_dict[i]) != pd.core.series.Series:
        asr_dict[i] = asr_dict[i].reset_index(drop = True)

#### Review dataset


In [242]:
ar_dict = {}
for i in ar.index.unique():
    ar_dict[i] = ar.loc[i, ['source','date','author', 'verified', 'vine', 'stars', 'pvotes', 'tvotes',
       'title', 'text', 'image', 'video']]

In [243]:
for i in ar_dict:
    if type(ar_dict[i]) != pd.core.series.Series:
        ar_dict[i] = ar_dict[i].sort_values('date').reset_index()
        ar_dict[i].drop_duplicates(inplace = True)
        ar_dict[i]['date'] = pd.to_datetime(ar_dict[i]['date'])
        ar_dict[i]['date'] = ar_dict[i]['date'].dt.strftime('%Y-%m-%d')
        ar_dict[i]['date'] = pd.to_datetime(ar_dict[i]['date'])

In [244]:
for i in ar_dict:
    if type(ar_dict[i]) != pd.core.series.Series:
        ar_dict[i] = ar_dict[i].reset_index(drop = True)

#### Price dataset

In [245]:
ap_dict = {}
for i in ap.index.unique():
    ap_dict[i] = ap.loc[i, ['id','buyboxtime','price',
       'high', 'low', 'stock', 'merchant','prime','subscribe',
       'pantry', 'addon', 'min', 'max', 'mapviolation', 'sellers',
       'Category_custom', 'Subcategory_custom']]

/Users/jiaxuewan/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [246]:
for i in ap_dict:
    if type(ap_dict[i]) != pd.core.series.Series:
        ap_dict[i] = ap_dict[i].drop(labels = 'id',axis = 1)
        ap_dict[i] = ap_dict[i].sort_values('buyboxtime').reset_index()
        ap_dict[i].drop_duplicates(inplace = True)
        ap_dict[i]['date'] = pd.to_datetime(ap_dict[i]['buyboxtime'])
        ap_dict[i]['date'] = ap_dict[i]['date'].dt.strftime('%Y-%m-%d')
        ap_dict[i]['date'] = pd.to_datetime(ap_dict[i]['date'])

### 2.1 Feature group: aggregated features


In [247]:
# Now that we have created two dictionaries, next we are going to do the aggregation
for i in asr_dict:
    if i in ar_dict:
        for j in asr_dict[i].index.unique():
            if type(ar_dict[i]) != pd.core.series.Series:
                asr_dict[i].loc[j, "stars_recent_10"] = ar_dict[i][ar_dict[i]['date'] <= asr_dict[i].loc[j, 'date']].tail(10)['stars'].mean()
    

In [248]:
for i in asr_dict:
    if i in ar_dict:
        for j in asr_dict[i].index.unique():
            if type(ar_dict[i]) != pd.core.series.Series:
                asr_dict[i].loc[j, 'stars_recent_oneweek'] = ar_dict[i][(ar_dict[i]['date'] <= asr_dict[i].loc[j, 'new_date']) & (ar_dict[i]['date'] >= (asr_dict[i].loc[j,'new_date'] - datetime.timedelta(days = 6)))]['stars'].mean()
                
                
                asr_dict[i].loc[j, 'stars_recent_onemonth'] = ar_dict[i][(ar_dict[i]['date'] <= 
                                                                          asr_dict[i].loc[j, 'new_date']) & (ar_dict[i]['date'] >= (asr_dict[i].loc[j,'new_date'] - datetime.timedelta(days = 29)))]['stars'].mean()
                
                
                asr_dict[i].loc[j, 'stars_avg'] = ar_dict[i][ar_dict[i]['date'] <= asr_dict[i].loc[j, 'new_date']]['stars'].mean()
                
                asr_dict[i].loc[j, 'review_acc'] = ar_dict[i][ar_dict[i]['date'] <= asr_dict[i].loc[j, 'new_date']]['date'].count()
                
                asr_dict[i].loc[j, 'reviewnum_oneweek'] = ar_dict[i][(ar_dict[i]['date'] <= 
                                                                          asr_dict[i].loc[j, 'new_date']) & (ar_dict[i]['date'] >= (asr_dict[i].loc[j,'new_date'] - datetime.timedelta(days = 6)))]['date'].count()
                
                
                asr_dict[i].loc[j, 'reviewnum_onemonth'] = ar_dict[i][(ar_dict[i]['date'] <= 
                                                                          asr_dict[i].loc[j, 'new_date']) & (ar_dict[i]['date'] >= (asr_dict[i].loc[j,'new_date'] - datetime.timedelta(days = 29)))]['date'].count()
                asr_dict[i].loc[j, 'author_acc'] = ar_dict[i][(ar_dict[i]['date'] <= asr_dict[i].loc[j,'new_date']) & 
                                                (ar_dict[i]['author'].str.contains('Customer') == False) & (ar_dict[i]['author'].str.contains('customer') == False)]['date'].count() /  ar_dict[i][ar_dict[i]['date'] <= asr_dict[i].loc[j,'new_date']]['date'].count()
                asr_dict[i].loc[j, 'author_acc_onemonth'] = ar_dict[i][(ar_dict[i]['date'] <= asr_dict[i].loc[j,'new_date']) & (ar_dict[i]['date'] >= (asr_dict[i].loc[j,'new_date'] - datetime.timedelta(days = 29))) &
                                                (ar_dict[i]['author'].str.contains('Customer') == False) & (ar_dict[i]['author'].str.contains('customer') == False)]['date'].count() /  ar_dict[i][(ar_dict[i]['date'] <= asr_dict[i].loc[j,'new_date']) & (ar_dict[i]['date'] >= (asr_dict[i].loc[j,'new_date'] - datetime.timedelta(days = 29)))]['date'].count()
                asr_dict[i].loc[j, 'num_1_month'] = ar_dict[i][(ar_dict[i]['date'] <= 
                                                                          asr_dict[i].loc[j, 'date']) & (ar_dict[i]['date'] >= (asr_dict[i].loc[j,'new_date'] - datetime.timedelta(days = 29))) & (ar_dict[i]['stars'] == 1)]['date'].count()
                asr_dict[i].loc[j, 'num_5_month'] = ar_dict[i][(ar_dict[i]['date'] <= 
                                                                          asr_dict[i].loc[j, 'date']) & (ar_dict[i]['date'] >= (asr_dict[i].loc[j,'new_date'] - datetime.timedelta(days = 29))) & (ar_dict[i]['stars'] == 5)]['date'].count()
                
    

In [87]:
asr_dict['B004ISI35Y']
                                            

KeyError: 'B004ISI35Y'

### 2.2 Text related Features:
* average word count

In [249]:
# of words in the reviews
# Generate a new column: average word count of review that happen on the same sales rank day
# for i in df_review2.index.unique():
#     df_review2.loc[i,'word_count'] = df_review2.loc[i,'text'].count(' ') + 1
  
for i in asr_dict:
    try:
        if i in ar_dict:
            for j in asr_dict[i].index.unique():
                if type(ar_dict[i]) != pd.core.series.Series:
                    asr_dict[i].loc[j, 'avg_word_count'] = ar_dict[i][ar_dict[i]['date'] <= asr_dict[i].loc[j, 'date']]['text'].apply(lambda x: x.count(' ')+1).mean()
                    asr_dict[i].loc[j, 'avg_word_count_10'] = ar_dict[i][ar_dict[i]['date'] <= asr_dict[i].loc[j, 'date']].tail(10)['text'].apply(lambda x: x.count(' ')+1).mean() 
                    asr_dict[i].loc[j, 'avg_word_count_onemonth'] = ar_dict[i][(ar_dict[i]['date'] <= 
                                                                          asr_dict[i].loc[j, 'new_date']) & (ar_dict[i]['date'] >= (asr_dict[i].loc[j,'new_date'] - datetime.timedelta(days = 29)))]['text'].apply(lambda x: x.count(' ')+1).mean()
    except:
        continue                                                                             

### 2.3 Features: reviewer status
* all: count of varified_account reviews 
* past one week: count of varified_account reviews

In [250]:
for i in asr_dict:
    if i in ar_dict:
        for j in asr_dict[i].index.unique():
            if type(ar_dict[i]) != pd.core.series.Series:
                asr_dict[i].loc[j, 'verified_acc'] = (ar_dict[i][(ar_dict[i]['date'] <= asr_dict[i].loc[j, 'date']) & (ar_dict[i]['verified'] == True)]['date'].count()) / (ar_dict[i][ar_dict[i]['date'] <= asr_dict[i].loc[j, 'date']]['date'].count()) 
            if type(ar_dict[i]) != pd.core.series.Series:
                asr_dict[i].loc[j, 'verified_acc_oneweek'] = (ar_dict[i][(ar_dict[i]['date'] <= 
                                                                          asr_dict[i].loc[j, 'new_date']) & (ar_dict[i]['date'] >= (asr_dict[i].loc[j,'new_date'] - datetime.timedelta(days = 6))) & (ar_dict[i]['verified'] == True)]['date'].count()) / (ar_dict[i][(ar_dict[i]['date'] <= 
                                                                          asr_dict[i].loc[j, 'new_date']) & (ar_dict[i]['date'] >= (asr_dict[i].loc[j,'new_date'] - datetime.timedelta(days = 6)))]['date'].count())
                asr_dict[i].loc[j, 'verified_acc_onemonth'] = (ar_dict[i][(ar_dict[i]['date'] <= 
                                                                          asr_dict[i].loc[j, 'new_date']) & (ar_dict[i]['date'] >= (asr_dict[i].loc[j,'new_date'] - datetime.timedelta(days = 29))) & (ar_dict[i]['verified'] == True)]['date'].count()) / (ar_dict[i][(ar_dict[i]['date'] <= 
                                                                          asr_dict[i].loc[j, 'new_date']) & (ar_dict[i]['date'] >= (asr_dict[i].loc[j,'new_date'] - datetime.timedelta(days = 29)))]['date'].count()) 
       

/Users/jiaxuewan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':


### 2.4 Price related features
* number of sellers
* price

In [167]:
ap_dict['B01HC4KIVW']['sellers'].unique()

KeyError: 'B01HC4KIVW'

In [251]:
for i in asr_dict:
    if i in ap_dict:
        for j in range(4,len(asr_dict[i].index)):
            if type(ap_dict[i]) != pd.core.series.Series:
                asr_dict[i].loc[j, 'latest_price'] = ap_dict[i][ap_dict[i]['buyboxtime'] <= asr_dict[i].loc[j, 'date']]['price'].iloc[-1]
                
                

In [252]:
for i in asr_dict:
    if i in ap_dict:
        for j in range(4,len(asr_dict[i].index)):
            if type(ap_dict[i]) != pd.core.series.Series:
                asr_dict[i].loc[j, 'first_price'] = ap_dict[i][ap_dict[i]['date'] == asr_dict[i].loc[j, 'new_date']]['price'].iloc[-1]
                
                

In [253]:
# fill NAs with the mean price
for i in asr_dict:
    asr_dict[i]['latest_price'] = asr_dict[i]['latest_price'].fillna(asr_dict[i]['latest_price'].mean())
    asr_dict[i]['first_price'] = asr_dict[i]['first_price'].fillna(asr_dict[i]['first_price'].mean())

In [ ]:
ap_dict['B004ISI35Y']

#### Remove problematic products with only 1 records

In [254]:
problem_products=[]
for i in asr_dict:
    if type(asr_dict[i]['category_rank1'])!= pd.core.series.Series:
        problem_products.append(i)
problem_products

[]

In [255]:
for item in problem_products:
    del asr_dict[item]

### 2.4 Features: log rank & delta rank

In [256]:
for i in asr_dict:
    asr_dict[i]['log_salesrank'] = asr_dict[i]['category_rank1'].apply(lambda x: math.log(x))
    asr_dict[i]['delta_salesrank'] = asr_dict[i]['log_salesrank'].diff()  

In [257]:
# Since we use the delta of log salesrank, the first row should be deleted
for i in asr_dict:
    asr_dict[i].drop(asr_dict[i].index[0],inplace = True)                    

#### We can tell many products have missing value so the calculated fields are missing

### 2.5 Data preparation for modeling
Models for different product groups
    * Large salesrank variation
    * Most reviews records

In [258]:
# To change the keys from product ids to numbers, preparing for the following loop.
new_dict_asr=dict((i,asr_dict[k]) for i,k in enumerate(sorted(asr_dict.keys())))

# pull out interested products to form a new dataframe

a = new_dict_asr[0]
for i in new_dict_asr:
    new_dict_asr[i]=new_dict_asr[i].reset_index()
    if new_dict_asr[i].loc[1,'id'] in common: 
        a = pd.concat([a,new_dict_asr[i]],ignore_index=True)


/Users/jiaxuewan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


In [259]:
a.columns

Index(['author_acc', 'author_acc_onemonth', 'avg_word_count',
       'avg_word_count_10', 'avg_word_count_onemonth', 'category_id1',
       'category_id2', 'category_id3', 'category_id4', 'category_name1',
       'category_name2', 'category_name3', 'category_name4', 'category_rank1',
       'category_rank2', 'category_rank3', 'category_rank4', 'date',
       'delta_rank', 'delta_salesrank', 'first_price', 'id', 'index',
       'latest_price', 'log_rank', 'log_salesrank', 'new_date', 'num_1_month',
       'num_5_month', 'review_acc', 'reviewnum_onemonth', 'reviewnum_oneweek',
       'source', 'stars_avg', 'stars_recent_10', 'stars_recent_onemonth',
       'stars_recent_oneweek', 'verified_acc', 'verified_acc_onemonth',
       'verified_acc_oneweek'],
      dtype='object')

In [260]:
# put our interested columns in the final dataframe.
final_df = a.reset_index(drop = True)[['id', 'date', 'category_name1',
       'category_rank1','log_rank', 'delta_rank',
       'stars_recent_10', 'stars_recent_oneweek',
       'stars_recent_onemonth', 'stars_avg', 'review_acc', 'reviewnum_oneweek',
       'reviewnum_onemonth', 'avg_word_count', 'verified_acc',
       'verified_acc_oneweek', 'log_rank', 'delta_rank','avg_word_count_10','num_1_month', 'num_5_month',
        'latest_price','first_price','author_acc','author_acc_onemonth','avg_word_count_onemonth','verified_acc_onemonth']]


# replace the NAs with 0
final_df = final_df.fillna(0)
temp=final_df
#final_df.to_csv('final_df.csv')

## 3. Modeling Part
1. Check missing values for calculated fields
2. Model construction
3. Model results
4. Interpretation of important variables

### 3.1 missing value check

In [261]:
complete_df= new_dict_asr[0]
for i in new_dict_asr:
    new_dict_asr[i]=new_dict_asr[i].reset_index()
    complete_df= pd.concat([complete_df,new_dict_asr[i]],ignore_index=True)

/Users/jiaxuewan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


In [262]:
# get the number of missing data points per column
def missing_col(df):
    missing_values_count = round(df.isnull().sum()/complete_df.shape[0]*100,1)
    rm_list=missing_values_count[missing_values_count>35].index
    return(missing_values_count,rm_list)

missing_values_count1,rm_list1=missing_col(complete_df)
missing_values_count1

author_acc                   0.0
author_acc_onemonth          0.0
avg_word_count               0.0
avg_word_count_10            0.0
avg_word_count_onemonth      0.0
category_id1                 0.0
category_id2                85.7
category_id3               100.0
category_id4               100.0
category_name1               0.0
category_name2              85.7
category_name3             100.0
category_name4             100.0
category_rank1               0.0
category_rank2              85.7
category_rank3             100.0
category_rank4             100.0
date                         0.0
delta_rank                   0.0
delta_salesrank              0.0
first_price                  0.0
id                           0.0
index                        0.0
latest_price                 0.0
level_0                     14.3
log_rank                     0.0
log_salesrank                0.0
new_date                     0.0
num_1_month                  0.0
num_5_month                  0.0
review_acc

In [263]:
missing_values_count2,rm_list2=missing_col(complete_df)
#complete_df=complete_df.drop(rm_list,axis=1)
missing_values_count2.head()

author_acc                 0.0
author_acc_onemonth        0.0
avg_word_count             0.0
avg_word_count_10          0.0
avg_word_count_onemonth    0.0
dtype: float64

In [264]:
final_df['id'].unique()

array(['B000WB397I', 'B000YGO7HW', 'B0026I2HLO', 'B00FNK8UQS',
       'B010E9M3KG', 'B074QHZJ9K'], dtype=object)

### 3.2 Model construction

In [265]:
final_df.columns

Index(['id', 'date', 'category_name1', 'category_rank1', 'log_rank',
       'delta_rank', 'stars_recent_10', 'stars_recent_oneweek',
       'stars_recent_onemonth', 'stars_avg', 'review_acc', 'reviewnum_oneweek',
       'reviewnum_onemonth', 'avg_word_count', 'verified_acc',
       'verified_acc_oneweek', 'log_rank', 'delta_rank', 'avg_word_count_10',
       'num_1_month', 'num_5_month', 'latest_price', 'first_price',
       'author_acc', 'author_acc_onemonth', 'avg_word_count_onemonth',
       'verified_acc_onemonth'],
      dtype='object')

In [266]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
X = final_df[['stars_recent_10', 'stars_recent_oneweek', 'stars_recent_onemonth', 'stars_avg', 'review_acc', 
               'reviewnum_onemonth', 'avg_word_count', 'avg_word_count_10','verified_acc', 'verified_acc_oneweek','num_1_month', 'num_5_month',
        'latest_price','first_price','author_acc','author_acc_onemonth', 'avg_word_count_onemonth',
       'verified_acc_onemonth']]

y = final_df['delta_rank']
poly = PolynomialFeatures(interaction_only=True,include_bias = False)
X = poly.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

reg = LinearRegression().fit(X_train, y_train)
reg.score(X_train, y_train)

/Users/jiaxuewan/anaconda3/lib/python3.7/site-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.27234184767525793

### 3.3 Model results

In [40]:
from sklearn.metrics import mean_squared_error, r2_score
prediction=reg.predict(X_test)
# The coefficients
print('Coefficients: \n', reg.coef_)
# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(y_test, prediction))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(y_test, prediction))

Coefficients: 
 [[ 7.02843003e+01  3.84362061e+00 -7.73112876e+01  9.80289830e+01
  -4.39861261e-01 -6.16637248e+00 -3.68296887e-01 -5.71814836e-01
  -1.51561106e+02 -6.29204010e+00 -2.28301257e+01  4.05519098e+00
   1.17430523e+01  1.17430523e+01  2.21416498e+02 -1.01231438e-01
  -5.26738602e-01 -2.69054956e+00  2.49137626e-03 -3.18191649e-01
  -4.07273931e-01  6.33961287e-02 -3.13832998e+01  1.13984604e+00
  -8.00700588e-03  3.33375914e-01 -5.10725511e-01 -5.10725507e-01
  -1.65001003e+01 -3.43317597e-01  4.72596142e-02 -9.96236463e-04
  -5.18799650e-03 -1.30913310e-02 -1.53521198e-03 -1.56164795e+00
   5.20787766e-03 -1.37699997e-01  3.33911040e-03  3.33393628e-02
   3.33393632e-02 -5.50437125e-01  1.28333824e+00  1.50737687e-02
   1.33617050e-01  2.73852528e-01 -7.95440925e-02  2.13175069e+01
   9.53183496e-01 -3.88455262e-01 -6.14131284e-02  4.25092895e-01
   4.25092895e-01  4.84921977e+01 -4.06273280e-02  1.26987688e-01
   1.64242583e-01 -4.23120423e-02  8.53196682e+00 -1.5432268

In [267]:
final_df.drop_duplicates(inplace = True)

In [268]:
final_df

,id,date,category_name1,category_rank1,log_rank,delta_rank,stars_recent_10,stars_recent_oneweek,stars_recent_onemonth,stars_avg,...,delta_rank,avg_word_count_10,num_1_month,num_5_month,latest_price,first_price,author_acc,author_acc_onemonth,avg_word_count_onemonth,verified_acc_onemonth
0,B000WB397I,2019-08-10 19:42:36,Shredders,15.0,2.708050,-0.125163,4.9,4.750000,4.947368,4.583362,...,-0.125163,26.6,0.0,18.0,189.9800,189.9800,0.924029,0.947368,19.631579,1.0
1,B000WB397I,2019-08-11 19:27:15,Shredders,19.0,2.944439,0.236389,4.9,4.800000,4.950000,4.583648,...,0.236389,36.6,0.0,19.0,189.9800,189.9800,0.924081,0.950000,24.200000,1.0
2,B000WB397I,2019-08-12 19:19:32,Shredders,20.0,2.995732,0.051293,4.9,4.800000,4.950000,4.583648,...,0.051293,36.6,0.0,19.0,189.9800,189.9800,0.924081,0.950000,24.200000,1.0
3,B000WB397I,2019-08-13 20:37:41,Shredders,17.0,2.833213,-0.162519,4.9,4.800000,4.947368,4.583648,...,-0.162519,36.6,0.0,18.0,189.9800,189.9800,0.924081,0.947368,24.947368,1.0
4,B000WB397I,2019-08-14 19:53:12,Shredders,17.0,2.833213,0.000000,4.9,5.000000,4.947368,4.583648,...,0.000000,36.6,0.0,18.0,189.9800,189.9800,0.924081,0.947368,24.947368,1.0
5,B000WB397I,2019-08-15 19:53:45,Shredders,17.0,2.833213,0.000000,4.9,5.000000,4.944444,4.583648,...,0.000000,36.6,0.0,17.0,189.9800,189.9800,0.924081,0.944444,25.166667,1.0
6,B000WB397I,2019-08-16 21:01:17,Shredders,15.0,2.708050,-0.125163,4.9,5.000000,4.944444,4.583648,...,-0.125163,36.6,0.0,17.0,189.9800,189.9800,0.924081,0.944444,25.166667,1.0
7,B000WB397I,2019-08-17 22:09:17,Shredders,16.0,2.772589,0.064539,4.9,5.000000,4.941176,4.583648,...,0.064539,36.6,0.0,16.0,189.9800,189.9800,0.924081,0.941176,26.235294,1.0
8,B000WB397I,2019-08-18 19:36:14,Shredders,16.0,2.772589,0.000000,4.9,0.000000,4.941176,4.583648,...,0.000000,36.6,0.0,16.0,189.9800,189.9800,0.924081,0.941176,26.235294,1.0
9,B000WB397I,2019-08-19 22:11:40,Shredders,17.0,2.833213,0.060625,4.9,5.000000,4.941176,4.583791,...,0.060625,42.1,0.0,16.0,189.9800,189.9800,0.924107,0.941176,29.529412,1.0


In [269]:
final_df.to_csv('FOP_No2.csv')

### 3.4 conclusion